In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
DATA_PATH = "/content/drive/MyDrive/LoL dataset/"
SEED = 42

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device

In [30]:
base = pd.read_csv(f'{DATA_PATH}base_data_forreal.csv')

In [61]:
leagues = ["LCK","LEC", "LCS", "PCS", "VCS", "CBLOL", "WLDs", "MSI"]
base = base[base["league"].isin(leagues)]

In [62]:
teams = base[base["position"] == "team"].copy()

In [69]:
players = base[base["position"] != "team"].copy()

In [63]:
drop_list_team = [
    "participantid",
    "playername",
    'position',
    "champion",
    "teamkills",
    "teamdeaths",
    "firstbloodkill",
    "firstbloodassist",
    "firstbloodvictim",
    "dragons",
    "damageshare",
    "earnedgoldshare",
    "total cs",
    "monsterkillsownjungle",
    "monsterkillsenemyjungle",
    "goldat20",
    "xpat20",
    "csat20",
    "opp_goldat20",
    "opp_xpat20",
    "opp_csat20",
    "golddiffat20",
    "xpdiffat20",
    "csdiffat20",
    "killsat20",
    "assistsat20",
    "deathsat20",
    "opp_killsat20",
    "opp_assistsat20",
    "opp_deathsat20",
    "goldat25",
    "xpat25",
    "csat25",
    "opp_goldat25",
    "opp_xpat25",
    "opp_csat25",
    "golddiffat25",
    "xpdiffat25",
    "csdiffat25",
    "killsat25",
    "assistsat25",
    "deathsat25",
    "opp_killsat25",
    "opp_assistsat25",
    "opp_deathsat25"
]


In [70]:
player_drop_list = [
    "participantid",
    "pick1",
    "pick2",
    "pick3",
    "pick4",
    "pick5",
    "teamkills",
    "teamdeaths",
    "firstdragon",
    "dragons",
    "opp_dragons",
    "elementaldrakes",
    "opp_elementaldrakes",
    "infernals",
    "mountains",
    "clouds",
    "oceans",
    "chemtechs",
    "hextechs",
    "dragons (type unknown)",
    "elders",
    "opp_elders",
    "firstherald",
    "heralds",
    "opp_heralds",
    "void_grubs",
    "opp_void_grubs",
    "firstbaron",
    "firsttower",
    "towers",
    "opp_towers",
    "firstmidtower",
    "firsttothreetowers",
    "turretplates",
    "opp_turretplates",
    "gspd",
    "gpr",
    "monsterkillsownjungle",
    "monsterkillsenemyjungle",
    "goldat20",
    "xpat20",
    "csat20",
    "opp_goldat20",
    "opp_xpat20",
    "opp_csat20",
    "golddiffat20",
    "xpdiffat20",
    "csdiffat20",
    "killsat20",
    "assistsat20",
    "deathsat20",
    "opp_killsat20",
    "opp_assistsat20",
    "opp_deathsat20",
    "goldat25",
    "xpat25",
    "csat25",
    "opp_goldat25",
    "opp_xpat25",
    "opp_csat25",
    "golddiffat25",
    "xpdiffat25",
    "csdiffat25",
    "killsat25",
    "assistsat25",
    "deathsat25",
    "opp_killsat25",
    "opp_assistsat25",
    "opp_deathsat25"
]


In [41]:
len(player_drop_list)

69

In [64]:
len(drop_list_team)

45

In [65]:
teams = teams.drop(columns=drop_list_team)

In [71]:
players = players.drop(columns=player_drop_list)

In [11]:
teams.columns

Index(['league', 'split', 'playoffs', 'date', 'game', 'patch', 'side',
       'position', 'teamname', 'ban1',
       ...
       'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15',
       'assistsat15', 'deathsat15', 'opp_killsat15', 'opp_assistsat15',
       'opp_deathsat15'],
      dtype='object', length=111)

In [66]:
teams.head(2)

,league,split,playoffs,date,game,patch,side,teamname,ban1,ban2,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
130,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,DRX,Diana,Caitlyn,...,510.0,4757.0,1540.0,1.0,4.0,7.0,1.0,1.0,1.0,4.0
131,LCK,Spring,0,2022-01-12 6:20,1,12.01,Red,BNK FEARX,Renekton,Lee Sin,...,511.0,-4757.0,-1540.0,-1.0,1.0,1.0,4.0,4.0,7.0,1.0


In [72]:
players.head(3)

,league,split,playoffs,date,game,patch,side,position,playername,teamname,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
120,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,top,Kingen,DRX,...,135.0,-202.0,-715.0,-8.0,0.0,0.0,1.0,0.0,1.0,0.0
121,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,jng,Pyosik,DRX,...,104.0,240.0,-271.0,-15.0,1.0,2.0,0.0,1.0,0.0,1.0
122,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,mid,Zeka,DRX,...,151.0,438.0,-204.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
players_train = players.drop('result', axis=1)
players_target = players['result']

In [67]:
teams_train = teams.drop('result', axis=1)
teams_target = teams['result']

In [68]:
teams.to_csv(f'{DATA_PATH}teams_1227.csv', index=False)
teams_target.to_csv(f'{DATA_PATH}teams_target_1227.csv', index=False)

In [74]:
players.to_csv(f'{DATA_PATH}players_1227.csv', index=False)
players_target.to_csv(f'{DATA_PATH}players_target_1227.csv', index=False)

#전처리 코드

In [77]:
# 행 출력 제한 해제
pd.set_option('display.max_rows', None)

In [78]:
players.isnull().sum()

,0
league,0
split,7075
playoffs,0
date,0
game,0
patch,0
side,0
position,0
playername,0
teamname,0


In [ ]:
columns_to_mean = [
    'damageshare', 'damagemitigatedperminute', 'earnedgoldshare',
    'total cs', 'minionkills', 'monsterkillsownjungle', 'monsterkillsenemyjungle',
    'cspm', 'goldat10', 'xpat10', 'csat10',
    'killsat10', 'assistsat10', 'deathsat10',
    'goldat15', 'xpat15', 'csat15', 'killsat15', 'assistsat15', 'deathsat15',
    'opp_goldat10', 'opp_xpat10', 'opp_csat10',
    'opp_killsat10', 'opp_assistsat10', 'opp_deathsat10',
    'goldat20', 'xpat20', 'csat20', 'opp_goldat20', 'opp_xpat20', 'opp_csat20',
    'killsat20', 'assistsat20', 'deathsat20', 'opp_killsat20', 'opp_assistsat20', 'opp_deathsat20',
    'goldat25', 'xpat25', 'csat25', 'opp_goldat25', 'opp_xpat25', 'opp_csat25',
    'killsat25', 'assistsat25', 'deathsat25', 'opp_killsat25', 'opp_assistsat25', 'opp_deathsat25',
    'opp_goldat15', 'opp_xpat15', 'opp_csat15', 'opp_killsat15', 'opp_assistsat15', 'opp_deathsat15',
     'barons', 'inhibitors'
]


In [ ]:
columns_to_zero = [ ]


In [ ]:
columns_to_unknown = [
    'split',
    'ban1'
    'ban2'
    'ban3',
    'ban4',
    'ban5'
]


- players

In [ ]:
for i in columns_to_unknown:
    players[i] = players[i].fillna('Unknown')

In [ ]:
for i in columns_to_zero:
    players[i] = players[i].fillna(0)

In [ ]:
for col in columns_to_mean:
    group_mean = players.groupby('playername')[col].transform('mean')
    players[col] = players[col].fillna(group_mean)
    players[col] = players[col].fillna(0)


- teams


In [ ]:
for i in columns_to_unknown:
    teams[i] = teams[i].fillna('Unknown')

In [ ]:
for i in columns_to_zero:
    teams[i] = teams[i].fillna(0)

In [ ]:
for col in columns_to_mean:
    group_mean = teams.groupby('teamname')[col].transform('mean')
    teams[col] = teams[col].fillna(group_mean)
    teams[col] = teams[col].fillna(0)
